In [15]:
import polars as pl
import markout

In [ ]:
# load pools
# TODO: can I totally skip this step?
pools = (
    # see pools.sql
    pl.read_csv('2024.5.2 uniswap pools for markout analysis.csv')
    # I need to filter out for WETH pairs only because I don't have USDC values for everything else
    # I'm filtering in the Python rather than the SQL so that it's extensible to USDC in the future.
    .filter(
        (pl.col('TOKEN0_ADDRESS') == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
        |
        (pl.col('TOKEN1_ADDRESS') == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
    )
    .with_columns(
        FEE = (pl.col('FEE')/100).cast(pl.Int8),
        # whether WETH is token0 or not
        # TODO: can I remove this and pass decimals to the markout?
        IS_WETH_TOKEN0 = (
            pl.when(pl.col('TOKEN0_ADDRESS') == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
            .then(True)
            .otherwise(False)
        )
    )
)

In [12]:
pools.head()

LIQUIDITY_POOL_ADDRESS,POOL_NAME,TOKEN0_ADDRESS,TOKEN0_DECIMALS,TOKEN1_ADDRESS,TOKEN1_DECIMALS,FEE,IS_WETH_TOKEN0
str,str,str,i64,str,i64,i8,bool
"""0x9359c87b38dd…","""uniswap: DPI-W…","""0x1494ca1f11d4…",18,"""0xc02aaa39b223…",18,30,false
"""0x919fa96e88d6…","""uniswap: WETH-…","""0xc02aaa39b223…",18,"""0xd533a949740b…",18,30,true
"""0x4628a0a564de…","""uniswap: RNDR-…","""0x6de037ef9ad2…",18,"""0xc02aaa39b223…",18,30,false
"""0xfcfdfc98062d…","""uniswap: WETH-…","""0xc02aaa39b223…",18,"""0xdef1ca1fb7fb…",18,100,true
"""0x0188c28ea32d…","""uniswap: WETH-…","""0xc02aaa39b223…",18,"""0xe410fbd0977c…",9,100,true


In [18]:
# FYI BOB/WETH is 0x3887e82dbdbe8ec6db44e6298a2d48af572a3b78
swaps = (
    #TODO: add datetime conversion (maybe?)
    pl.read_csv('all_swaps.csv', dtypes={'sqrt_price_x96':pl.String})
    .with_columns(
        pl.col('block_timestamp').str.strptime(pl.Datetime, format = '%Y-%m-%d %H:%M:%S %Z')
    )
    .sort(['liquidity_pool_address', 'block_number', 'log_index'])
)

In [23]:
# testing my code here

testing = (markout.addPrice(swaps)
 .with_columns(
   # adj_price is the price of token0 in terms of token1, i.e., how many token1 needed to buy one token0.
    adj_price = (
         (10 ** (pl.col('token1_decimals') - pl.col('token0_decimals')))
         /
         (pl.col('price'))
    )
 )
 .with_columns(
     markout_5m = pl.col('adj_price')
                    .last() # last value over...
                    .over(
                        pl.col('liquidity_pool_address'), # group by pool and...
                        pl.col('block_timestamp').dt.truncate('5m')
                    ) # 5 mins worth of 'block_timestamp'
                    .shift(-1)
 )
)

In [25]:
# liquidity_pool_address = testing['liquidity_pool_address'].unique().first()

liquidity_pool_address = testing.select(pl.first('liquidity_pool_address'))[0]

filtered_df = testing.filter(pl.col('liquidity_pool_address') == liquidity_pool_address)



In [28]:
(filtered_df
 .select(['block_timestamp', 'adj_price', 'markout_5m'])
 .write_csv(file='testing.csv')
)

In [ ]:
"""
    TODO:
    something like
    for pool in "pools", which is a pl.DataFrame
    get the pool address and chain
    call load_all_swaps, which returns a pl.DataFrame
    pass the df, TOKEN0_DECIMALS, TOKEN1_DECIMALS, IS_WETH_TOKEN0, and weth_prices to construct_markout
    call execute_markout, which returns a pl.DataFrame
    append the DataFrame to "results", a pl.DataFrame
"""

def process_pools(pools: pl.DataFrame) -> pl.DataFrame:
    # Initialize an empty list to store the results
    results = []

    for pool in pools.iter_rows(named=True):

        pool_address = pool['LIQUIDITY_POOL_ADDRESS']
        chain = 'ethereum' 
        # TODO: lint - these should be lowercase
        TOKEN0_DECIMALS = pool['TOKEN0_DECIMALS']
        TOKEN1_DECIMALS = pool['TOKEN1_DECIMALS']
        IS_WETH_TOKEN0 = pool['IS_WETH_TOKEN0']
        fee = pool['FEE']

        # Load all swaps for the current pool
        swaps_df = markout.load_all_swaps(pool_address, chain)

        # Construct the markout DataFrame
        markout_df = markout.construct_markout(swaps_df, TOKEN0_DECIMALS, TOKEN1_DECIMALS, IS_WETH_TOKEN0)

        # Execute the markout calculation
        markout_result = markout.execute_markout(markout_df)

        # Add pool information to the result
        markout_result['pool_address'] = pool_address
        markout_result['chain'] = chain
        markout_result['fee'] = fee

        # Append the result to the list
        results.append(markout_result)

    # Convert the list of results to a Polars DataFrame
    result_df = pl.from_dicts(results)

    return result_df

# Assuming you have a DataFrame called 'pools' with pool information
# and a DataFrame called 'weth_prices' with WETH price data
result_df = process_pools(pools)

# TODO: then plot. x axis as volume, y axis as markout, can plot fee as dot color.

In [ ]:
bad_results = (result_df.explode('total_volume', 'total_markout')
      .filter(
          (pl.col('total_volume') != 0.0) &
          (pl.col('total_markout') > -508827)
      )
      )

In [ ]:
bad_results.select(["total_volume", "total_markout"]).plot.scatter(x="total_volume", y="total_markout")

In [ ]:
result_df.explode(['total_volume', 'total_markout']).head()
# result_df.explode(['total_volume_usd', 'total_markout_usd']).head().write_csv('results.csv')

In [ ]:
exploded_df = result_df.explode(['total_volume', 'total_markout'])

exploded_df.select(["total_volume", "total_markout"]).plot.scatter(x="total_volume", y="total_markout")